In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

In [4]:
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
x_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

x_train = df_train.drop(columns=['SalePrice'])
y_train = df_train['SalePrice']

In [5]:
na_threshold = 0.8
train_na_ratios = x_train.isna().mean()
test_na_ratios = x_test.isna().mean()
na_cols_to_drop = train_na_ratios[train_na_ratios > na_threshold].index
x_train = x_train.drop(columns=na_cols_to_drop)
x_test = x_test.drop(columns=na_cols_to_drop)
print("Dropped ", len(na_cols_to_drop) ," columns:", list(na_cols_to_drop), "x_train/test.shape(): ", x_train.shape)

Dropped  4  columns: ['Alley', 'PoolQC', 'Fence', 'MiscFeature'] x_train/test.shape():  (1460, 76)


In [6]:
train_ids = x_train.pop('Id')
test_ids = x_test.pop('Id')

In [7]:
x_train.drop('GarageYrBlt', axis=1, inplace=True)
x_test.drop('GarageYrBlt', axis=1, inplace=True)

In [8]:
x_train.drop(columns=['PoolArea'], inplace=True)
x_test.drop(columns=['PoolArea'], inplace=True)

In [9]:
cat_cols = [col for col in x_train.columns if x_train[col].dtype == 'object']
num_cols = [col for col in x_train.columns if x_train[col].dtype != 'object']

print(f"Categorical columns ({len(cat_cols)}): {cat_cols}")
print(f"Numerical columns ({len(num_cols)}): {num_cols}")

Categorical columns (39): ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']
Numerical columns (34): ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch'

In [10]:
s = x_train[cat_cols].nunique()

In [11]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Train the model on non-null values from x_train
train = x_train[x_train['LotFrontage'].notna()]
model = LinearRegression()
model.fit(np.log1p(train['LotArea'].values.reshape(-1, 1)), train['LotFrontage'])

# Fill missing values in x_train
predict_train = x_train[x_train['LotFrontage'].isna()]
predicted_frontage_train = model.predict(np.log1p(predict_train['LotArea'].values.reshape(-1, 1)))
x_train.loc[x_train['LotFrontage'].isna(), 'LotFrontage'] = predicted_frontage_train

# Fill missing values in x_test using the same model
predict_test = x_test[x_test['LotFrontage'].isna()]
predicted_frontage_test = model.predict(np.log1p(predict_test['LotArea'].values.reshape(-1, 1)))
x_test.loc[x_test['LotFrontage'].isna(), 'LotFrontage'] = predicted_frontage_test

In [12]:
x_train['MasVnrArea'] = x_train['MasVnrArea'].fillna(0)
x_test['MasVnrArea'] = x_test['MasVnrArea'].fillna(0)

In [13]:
cols_to_drop = ['Street', 'Utilities', 'Condition2', 'RoofMatl']
x_train.drop(columns=cols_to_drop, inplace=True, errors='ignore')
x_test.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print("\nColumns dropped successfully.")


Columns dropped successfully.


In [14]:
x_train['GarageCond_Problem'] = (x_train['GarageCond'] != 'TA').astype(int)
x_test['GarageCond_Problem'] = (x_test['GarageCond'] != 'TA').astype(int)

x_train.drop(columns=['GarageCond'], inplace=True)
x_test.drop(columns=['GarageCond'], inplace=True)

In [15]:
cat_cols = [col for col in cat_cols if col not in cols_to_drop]

In [16]:
threshold = 3
woe_columns = list(s[s > 3].index)
one_hot_columns = list(s[s <= 3].index)

In [17]:
valid_woe_columns = [col for col in woe_columns if col in x_train.columns]
x_train[valid_woe_columns].mode().T[0].to_dict()

{'MSZoning': 'RL',
 'LotShape': 'Reg',
 'LandContour': 'Lvl',
 'LotConfig': 'Inside',
 'Neighborhood': 'NAmes',
 'Condition1': 'Norm',
 'BldgType': '1Fam',
 'HouseStyle': '1Story',
 'RoofStyle': 'Gable',
 'Exterior1st': 'VinylSd',
 'Exterior2nd': 'VinylSd',
 'ExterQual': 'TA',
 'ExterCond': 'TA',
 'Foundation': 'PConc',
 'BsmtQual': 'TA',
 'BsmtCond': 'TA',
 'BsmtExposure': 'No',
 'BsmtFinType1': 'Unf',
 'BsmtFinType2': 'Unf',
 'Heating': 'GasA',
 'HeatingQC': 'Ex',
 'Electrical': 'SBrkr',
 'KitchenQual': 'TA',
 'Functional': 'Typ',
 'FireplaceQu': 'Gd',
 'GarageType': 'Attchd',
 'GarageQual': 'TA',
 'SaleType': 'WD',
 'SaleCondition': 'Normal'}

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, woe_columns, one_hot_columns):
        self.woe_columns = woe_columns
        self.one_hot_columns = one_hot_columns

    def fit(self, X, y):
        # Store mode values for WOE columns
        self.woe_columns_fill_na = X[self.woe_columns].mode().T[0].to_dict()
        
        # Calculate WOE and IV
        df_woe = X[self.woe_columns].copy()
        df_woe['SalePrice'] = y
        
        self.woe_mappings = {}
        self.iv_values = {}
        
        for col in self.woe_columns:
            # Group by category and calculate metrics
            groups = df_woe.groupby(col)['SalePrice'].agg(['count', 'mean'])
            groups['n_pos'] = groups['mean'] * groups['count']
            groups['n_neg'] = groups['count'] - groups['n_pos']
            
            # Calculate proportions
            total_pos = groups['n_pos'].sum()
            total_neg = groups['n_neg'].sum()
            groups['prop_pos'] = groups['n_pos'] / total_pos
            groups['prop_neg'] = groups['n_neg'] / total_neg
            
            # Calculate WOE and IV
            groups['woe'] = np.log(groups['prop_pos'] / groups['prop_neg'])
            groups['iv'] = (groups['prop_pos'] - groups['prop_neg']) * groups['woe']
            
            # Clean infinite/NA values
            groups.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
            
            # Store mappings
            self.woe_mappings[col] = groups['woe'].to_dict()
            self.iv_values[col] = groups['iv'].sum()

        return self

    def transform(self, X):
        X_transformed = X.copy()
        
        # Apply WOE encoding
        for col in self.woe_columns:
            new_col = f'{col}_woe'
            # Map values and handle NAs in one step
            X_transformed[new_col] = (
                X_transformed[col]
                .map(self.woe_mappings[col])
                .fillna(self.woe_mappings[col].get(self.woe_columns_fill_na[col], 0))
            )
        
        # One-hot encode remaining categoricals
        X_transformed = pd.get_dummies(
            X_transformed,
            columns=self.one_hot_columns,
            drop_first=True,
            dummy_na=True
        )
        
        # Drop original columns safely
        cols_to_drop = [c for c in (self.woe_columns + self.one_hot_columns) 
                       if c in X_transformed.columns]
        X_transformed = X_transformed.drop(columns=cols_to_drop)
        
        # Final NA clean-up (shouldn't be needed if dummy_na=True)
        return X_transformed.fillna(0)

In [19]:
valid_one_hot_columns = [col for col in one_hot_columns if col in x_train.columns]
preprocessor = CustomPreprocessor(
    woe_columns=valid_woe_columns,
    one_hot_columns=valid_one_hot_columns
)

In [20]:
X_train_t = preprocessor.fit_transform(x_train, y_train)

In [21]:
X_test_t = preprocessor.transform(x_test)

In [22]:
X_corr = X_train_t.copy()
X_corr['SalePrice'] = y_train

In [23]:
available_cols = [col for col in X_corr.columns if col != 'SalePrice']
corr_matrix = X_corr[available_cols + ['SalePrice']].corr().abs()

In [24]:
features_to_drop =  ['GarageArea', 'Exterior1st_woe', '1stFlrSF', 'TotRmsAbvGrd', 
                     'SaleCondition_woe', 'MasVnrType_BrkFace']
X_filtered = X_train_t.drop(columns=features_to_drop)
X_test_new = X_test_t.drop(columns=features_to_drop)
print(f"Original shape: {X_train_t.shape}, New shape: {X_filtered.shape}")

Original shape: (1460, 78), New shape: (1460, 72)


In [25]:
!pip install dagshub
import dagshub
dagshub.init(repo_owner='tvani2', repo_name='assn1', mlflow=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 5.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b7e19dc8-084d-4796-90dc-c6d8886eb173&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=cdafe2df94449bd6cd97a82a8e3e75e8dc4fb59804357b6e00593722d6fd80d5




Output()

Accessing as tvani2

Initialized MLflow to track repo "tvani2/assn1"

Repository tvani2/assn1 initialized!

In [26]:
!pip install mlflow
import mlflow
import mlflow.sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 62.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 79.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00


In [30]:
import mlflow
logged_model = 'runs:/1b83b51f6fb5469f86c3b884f4cf4dc7/final_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test_new))

2025/04/11 18:11:36 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 3.1.1, required: cloudpickle==3.1.0)
 - psutil (current: 7.0.0, required: psutil==5.9.5)
 - scipy (current: 1.15.2, required: scipy==1.13.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/04/11 18:11:36 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.12`, differs from the version of Python that is currently running, `Python 3.11.11`, and may be incompatible


array([117942.59097901, 166893.76533229, 186148.57533535, ...,
       181773.40115174, 129674.85763322, 211297.42781996])

In [28]:
mlflow.pyfunc.get_model_dependencies(logged_model)

2025/04/11 18:05:46 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmpat9aek_s/final_model/requirements.txt'.


'/tmp/tmpat9aek_s/final_model/requirements.txt'

In [29]:
%pip install -r /tmp/tmpat9aek_s/final_model/requirements.txt'

/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file
Note: you may need to restart the kernel to use updated packages.


In [32]:
import mlflow
import pandas as pd

# Load model as a PyFuncModel
logged_model = 'runs:/1b83b51f6fb5469f86c3b884f4cf4dc7/final_model'
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame
# Assuming X_test_new is your test data
y_pred = loaded_model.predict(pd.DataFrame(X_test_new))

# Save predictions to a CSV file for Kaggle submission
output = pd.DataFrame({
    'Id': test_ids,  # Make sure these are the original IDs
    'SalePrice': y_pred  # Use the predictions from the model
})

output.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")

2025/04/11 18:15:33 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 3.1.1, required: cloudpickle==3.1.0)
 - psutil (current: 7.0.0, required: psutil==5.9.5)
 - scipy (current: 1.15.2, required: scipy==1.13.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/04/11 18:15:33 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.12`, differs from the version of Python that is currently running, `Python 3.11.11`, and may be incompatible


Predictions saved to submission.csv
